# Model 3. CatBoost + Optuna Tuning

범주형 데이터 처리에 특화된 **CatBoost**를 사용하여 모델을 학습시킵니다.
과적합을 방지하는 기능이 강력하며, 하이퍼파라미터에 덜 민감하여 기본 설정으로도 좋은 성능을 보여줍니다.

### 진행 순서
1.  데이터 로드 (train_enriched.csv, test_enriched.csv)
2.  데이터 인코딩 (Label Encoding)
3.  Optuna를 이용한 하이퍼파라미터 최적화
4.  최적의 파라미터로 최종 모델 학습
5.  결과 예측 및 제출 파일 생성 (submission_cat_optuna.csv)

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
from catboost import CatBoostRegressor
import optuna
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

warnings.filterwarnings('ignore')

# 1. 데이터 로드 및 준비
PREPROCESS_DIR = './data_preprocess'
SUBMISSION_DIR = './open_track1'

TRAIN_PATH = os.path.join(PREPROCESS_DIR, 'train_enriched.csv')
TEST_PATH = os.path.join(PREPROCESS_DIR, 'test_enriched.csv')
SUBMISSION_PATH = os.path.join(SUBMISSION_DIR, 'sample_submission.csv')

print("데이터 로딩 중...")
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

# 인코딩
cat_features = ['type_name', 'prev_type_name']
le = LabelEncoder()

for col in cat_features:
    all_values = pd.concat([train_df[col].astype(str), test_df[col].astype(str)]).unique()
    le.fit(all_values)
    train_df[col] = le.transform(train_df[col].astype(str))
    test_df[col] = le.transform(test_df[col].astype(str))

# 피처 선택
feature_cols = [
    'start_x', 'start_y', 'type_name', 'team_id', 'time_seconds',
    'prev_type_name', 'prev_end_x', 'prev_end_y',
    'dist_to_goal', 'angle_to_goal', 'dist_to_center'
]
target_cols = ['end_x', 'end_y']

X = train_df[feature_cols]
y = train_df[target_cols]
X_test = test_df[feature_cols]
test_ids = test_df['game_episode']

# 튜닝용 데이터 분리 (8:2)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"학습 데이터 준비 완료: {X_train.shape}")

# 2. Optuna를 이용한 하이퍼파라미터 튜닝
def objective(trial):
    # 튜닝할 파라미터 범위 설정
    params = {
        'loss_function': 'RMSE',
        'verbose': False,
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'random_seed': 42
    }
    
    # X축 예측 모델
    model_x = CatBoostRegressor(**params)
    model_x.fit(X_train, y_train['end_x'])
    pred_x = model_x.predict(X_val)
    
    # Y축 예측 모델
    model_y = CatBoostRegressor(**params)
    model_y.fit(X_train, y_train['end_y'])
    pred_y = model_y.predict(X_val)
    
    # 평균 RMSE 계산
    rmse_x = np.sqrt(mean_squared_error(y_val['end_x'], pred_x))
    rmse_y = np.sqrt(mean_squared_error(y_val['end_y'], pred_y))
    
    return (rmse_x + rmse_y) / 2

print("\nOptuna 튜닝 시작 (20회 시도)...")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

print(f"\nBest Trial: {study.best_trial.value:.4f}")
print("Best Params:", study.best_params)

# 3. 최적 파라미터로 최종 학습 및 예측
print("\n최적 파라미터로 전체 데이터 재학습 중...")

best_params = study.best_params
best_params['loss_function'] = 'RMSE'
best_params['verbose'] = False
best_params['random_seed'] = 42

# 전체 데이터로 학습
final_model_x = CatBoostRegressor(**best_params)
final_model_y = CatBoostRegressor(**best_params)

final_model_x.fit(X, y['end_x'])
final_model_y.fit(X, y['end_y'])

# 예측
pred_x = final_model_x.predict(X_test)
pred_y = final_model_y.predict(X_test)

# Clipping (경기장 규격 보정)
pred_x = np.clip(pred_x, 0, 105)
pred_y = np.clip(pred_y, 0, 68)

# 4. 제출 파일 생성
submission_df = pd.DataFrame({
    'game_episode': test_ids,
    'end_x': pred_x,
    'end_y': pred_y
})

sample_submission = pd.read_csv(SUBMISSION_PATH)
final_submission = pd.merge(sample_submission[['game_episode']], submission_df, on='game_episode', how='left')
final_submission.fillna(50.0, inplace=True)

save_path = 'submission_cat_optuna.csv'
final_submission.to_csv(save_path, index=False)

print(f"\nCatBoost 튜닝 완료! 결과 저장됨: {save_path}")

c:\Users\jwk72\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-12-02 14:28:20,814] A new study created in memory with name: no-name-a065238a-6bc5-4c1c-bded-46aed816c107


데이터 로딩 중...
학습 데이터 준비 완료: (12348, 11)

Optuna 튜닝 시작 (20회 시도)...


[I 2025-12-02 14:28:23,597] Trial 0 finished with value: 13.713961111221387 and parameters: {'iterations': 253, 'learning_rate': 0.1777800739886037, 'depth': 7, 'l2_leaf_reg': 1.6212951527004487, 'subsample': 0.6801982652228796}. Best is trial 0 with value: 13.713961111221387.
[I 2025-12-02 14:28:29,034] Trial 1 finished with value: 14.185141123397061 and parameters: {'iterations': 562, 'learning_rate': 0.2870647513405434, 'depth': 7, 'l2_leaf_reg': 3.2386004073919663, 'subsample': 0.754000965932448}. Best is trial 0 with value: 13.713961111221387.
[I 2025-12-02 14:28:30,624] Trial 2 finished with value: 13.508666578775408 and parameters: {'iterations': 257, 'learning_rate': 0.07742709050617362, 'depth': 5, 'l2_leaf_reg': 7.038044655266594, 'subsample': 0.779465674493662}. Best is trial 2 with value: 13.508666578775408.
[I 2025-12-02 14:28:33,997] Trial 3 finished with value: 13.505341154434241 and parameters: {'iterations': 143, 'learning_rate': 0.10367844403300458, 'depth': 9, 'l2_le


Best Trial: 13.4563
Best Params: {'iterations': 200, 'learning_rate': 0.06563154654477267, 'depth': 8, 'l2_leaf_reg': 7.212929810822191, 'subsample': 0.9372108026847834}

최적 파라미터로 전체 데이터 재학습 중...

CatBoost 튜닝 완료! 결과 저장됨: submission_cat_optuna.csv
